# Flower Server 코드
- server 실행 시 strategy를 통해 다양한 설정값을 지정
- output으로 각 라운드의 집계된 손실값과 성능 지표가 저장됨
    - 성능 지표(아래 셀 에선 accuracy)는 evaluate_metrics_aggregation_fn을 통해 집계됨

## 만약 아래 셀을 실행하고 연합학습 완료 전에 종료 시 '반드시 커널 재시작을 통해 초기화 해야 함'

In [1]:
import flwr as fl
import os
import numpy as np
# 동시 9명 연합학습을 위해 cpu 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

fraction_fit=1
fraction_eval=1
min_fit_clients=3
min_eval_clients=3
min_available_clients=3
num_rounds=10

def evaluate_metrics_aggregation_fn(eval_metrics):
    data_len = sum([num for num, met in eval_metrics])
    acc = sum([num*met['accuracy'] for num, met in eval_metrics])/data_len
    return {'accuracy' : acc}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=fraction_fit,                    # 훈련을 위해서 사용 가능한 클라이언트의 100% 이용
    fraction_evaluate=fraction_eval,              # 평가를 위해서 사용 가능한 클라이언트의 100% 이용
    min_fit_clients=min_fit_clients,              # 훈련을 위해서는 적어도 5개 이상의 클라이언트가 필요
    min_evaluate_clients=min_eval_clients,        # 평가를 위해서는 적어도 5개 이상의 클라이언트가 필요
    min_available_clients=min_available_clients,  # 사용 가능한 클라이언트의 수가 5 될 때까지 대기
    evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
)

print('server start!')
output = fl.server.start_server(config=fl.server.ServerConfig(num_rounds=num_rounds), strategy=strategy)
output

2023-08-10 20:55:19.550167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 20:55:19.646241: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 20:55:19.671115: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-10 20:55:20.064910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

server start!


INFO flwr 2023-08-10 20:55:39,329 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-08-10 20:55:39,330 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-10 20:55:39,331 | server.py:101 | FL starting
DEBUG flwr 2023-08-10 20:55:47,722 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 3)
DEBUG flwr 2023-08-10 20:55:49,334 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-10 20:55:49,341 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-10 20:55:49,343 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 3)
DEBUG flwr 2023-08-10 20:55:49,735 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
DEBUG flwr 2023-08-10 20:55:49,736 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 3)
DEBUG flwr 2023-08-10 20:55:50,930 | server.py:232 | fit_round 2 received 3 results and 0 failures
DEBUG flwr 2023-08-10 20:55:50,936 | se

History (loss, distributed):
	round 1: 0.25994672174453737
	round 2: 0.19096916469335556
	round 3: 0.15640759750008584
	round 4: 0.13673105250597
	round 5: 0.1209870516717434
	round 6: 0.11128747137784958
	round 7: 0.10222893356829882
	round 8: 0.0963453742697835
	round 9: 0.09085558602660894
	round 10: 0.08535868340283632
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.928000002002716), (2, 0.9436000126481057), (3, 0.9538000010371208), (4, 0.9591000057578087), (5, 0.9641000053405762), (6, 0.9664000087857246), (7, 0.9684000042319297), (8, 0.9720000284075737), (9, 0.9725999884366989), (10, 0.9740999801635742)]}